In [1]:
#import key libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [6]:
!pip install geocoder

    100% |████████████████████████████████| 102kB 17.0MB/s a 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [7]:
import geocoder

In [8]:
#define the url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [9]:
#pass the url through the requests.get function to call for the textual data from Wikipedia
wiki_url = requests.get(url).text

In [10]:
#pass the text data through the BeatifulSoup function to organize the information
soup = BeautifulSoup(wiki_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":900271985,"wgRevisionId":900271985,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June",

In [11]:
#scrape the html data to find the class that contains the postal, neighborhood and borough data
Neighborhood_table = soup.find('table',{'class':'wikitable sortable'})
Neighborhood_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [12]:
#scan for the data within the 'tr' heading to identify the postal code data
tr_rows = Neighborhood_table.findAll('tr')
tr_rows_skip = tr_rows[1:]

In [13]:
#validating my scan was succsseful at scanning the data
tr_rows_skip[5]

<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>

In [21]:
#Defining my column headers
columns=['Postal Code','Borough','Neighborhood']

In [22]:
FSA_df = pd.DataFrame([], columns =  columns) #creating the dataframe and assigning the columns headers

for row in tr_rows_skip: #looping through the tr_rows_skip object and appending each postal code, neighborhood and borough to a list 'FSA'

    FSA = []
    FSA.append(row.find_all('td')[0].get_text())
    FSA.append(row.find_all('td')[1].get_text())
    FSA.append(row.find_all('td')[2].get_text())
    #print(FSA)
    FSA_df = FSA_df.append(pd.DataFrame([FSA], columns = columns)) #appending 'FSA' list to the FSA_df dataframe
    #print(FSA_df)

FSA1_df = FSA_df[(FSA_df['Borough'] !='Not assigned')] #cleaning up the data and grouping neighborhoods
FSA1_df.reset_index(drop=True, inplace=True)
FSA1_df['Neighborhood'] = FSA1_df['Neighborhood'].map(lambda x: str(x).lstrip('\n').rstrip('\n'))
FSA2_df = FSA1_df.groupby('Postal Code').agg({'Borough':'first', 
                             'Neighborhood': ', '.join, 
                             }).reset_index()
FSA2_df['Neighborhood'] = np.where(FSA2_df['Neighborhood']=='Not assigned',FSA2_df['Borough'], FSA2_df['Neighborhood'])
FSA2_df

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal Code,Neighborhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
5,M1J,Scarborough Village,Scarborough
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough
9,M1N,"Birch Cliff, Cliffside West",Scarborough


In [16]:
FSA2_df.shape

(103, 3)

In [17]:
#FSA2_df.to_csv('Toronto_Neighborhoods.csv', index=False)

In [18]:
# geocoder did not return anything and kept looping

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(FSA2_df['PostalCode']))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: 

In [36]:
import io
import requests
url = "https://cocl.us/Geospatial_data"
s = requests.get(url).content
c = pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [39]:
C2 = c.merge(FSA2_df, on='Postal Code', how='outer')
C3 = C2[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
C3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [41]:
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [42]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [43]:
import folium

In [77]:
C4 = C3[C3['Borough'].str.contains("Toronto")] #filter on boroughs that only contain the word "Toronto"
C4.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [60]:
map_toronto = folium.Map(location=[43.70011, -79.4163], zoom_start=13) #creating a folium map to visualize the boroughs and their neighborhoods

for lat, lng, borough, neighborhood in zip(C4['Latitude'], C4['Longitude'], C4['Borough'], C4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

In [68]:
# set number of clusters
kclusters = 5

toronto_C4_clustering = C4.drop(columns=['Neighborhood', 'Borough', 'Postal Code'], axis=1) #define features for clustering

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_C4_clustering) #fit model

kmeans.labels_[0:10]

array([3, 3, 3, 3, 1, 1, 1, 1, 1, 1], dtype=int32)

In [78]:
# add clustering labels
C4.insert(0, 'Cluster Labels', kmeans.labels_)

In [80]:
C4.head()

,Cluster Labels,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,3,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,3,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,3,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,3,M4M,East Toronto,Studio District,43.659526,-79.340923
44,1,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [81]:
toronto_clusters = folium.Map(location=[43.70011, -79.4163], zoom_start=13) #create map of Toronto borough clusters

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(C4['Latitude'], C4['Longitude'], C4['Neighborhood'], C4['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_clusters)
       
toronto_clusters